In [1]:
import rclpy
from rclpy.node import Node
from controller_interface_msgs.msg import ControllerConfig
from controller_interface_msgs.msg import SetDesiredJointValue
from controller_interface_msgs.msg import SetDesiredTaskValue
from controller_interface_msgs.msg import UpdateGain
from opt_ctrl_msgs.msg import SetReference
from robot_state_msgs.msg import ExtendedState
from mcp_ep_msgs.msg import MujocoApplyForceIndex
import time
import numpy as np

import subprocess

In [2]:
class ControllerInterfaceNode(Node):
    
    def __init__(self):
        super().__init__('controller_interface_node')
        self.config_publisher_ = self.create_publisher(ControllerConfig, 'controller_config', 10)
        self.desired_joint_value_publisher_ = self.create_publisher(SetDesiredJointValue, 'desired_joint_value', 10)
        self.desired_task_value_publisher_ = self.create_publisher(SetDesiredTaskValue, 'desired_task_value', 10)
        self.update_gain_publisher_ = self.create_publisher(UpdateGain, 'update_gain', 10)
        self.mujoco_apply_force_publisher = self.create_publisher(MujocoApplyForceIndex, 'mujoco_apply_force_index', 10)
        
    def change_conroller(self, controller_name):
        msg = ControllerConfig()
        msg.controller_selector = controller_name
        self.config_publisher_.publish(msg)

    def set_desired_joint_value(self, q_d, dq_d, ddq_d, set_traj):
        msg = SetDesiredJointValue()
        msg.q_d = q_d
        msg.dq_d = dq_d
        msg.ddq_d = ddq_d
        msg.set_traj = set_traj
        self.desired_joint_value_publisher_.publish(msg)

    def set_desired_task_value(self, pos_d, rot_d, quat_d, set_traj):
        msg = SetDesiredTaskValue()
        msg.pos_d = pos_d
        msg.rot_d = rot_d
        msg.quat_d = quat_d
        msg.set_traj = set_traj
        self.desired_task_value_publisher_.publish(msg)    

    def update_gain(self, is_update):
        msg = UpdateGain()
        msg.update = is_update
        self.update_gain_publisher_.publish(msg)
        
    def mujoco_apply_force_index_pub(self, face_indices, link_indicies, forces_norm, k_env):
        msg = MujocoApplyForceIndex()
        msg.face_index = face_indices
        msg.link_index = link_indicies
        msg.force_norm = forces_norm
        msg.k_env = k_env
        self.mujoco_apply_force_publisher.publish(msg)
        
    def mujoco_apply_force_index_clear(self):
        msg = MujocoApplyForceIndex()
        msg.face_index = []
        msg.link_index = []
        msg.force_norm = []
        msg.k_env = 0.0
        self.mujoco_apply_force_publisher.publish(msg)
        
        
class MPCControllerInterfaceNode(Node):
    
    def __init__(self):
        super().__init__('mpc_controller_interface_node')
        self.set_reference_publisher_ = self.create_publisher(SetReference, 'set_reference', 10)
        timer_period = 0.001  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.initial_state = None

    def set_initial_state(self, initial_state: ExtendedState):
        # subscribe initial ExtendedState topic
        self.initial_state = initial_state
        self.init_t = self.get_clock().now().nanoseconds / 1e9
        
        
    def timer_callback(self):
        msg = self.circular_trajectory()
        self.set_reference_publisher_.publish(msg)
        
    def circular_trajectory(self, radius=0.05, frequency=1.0):
        msg = SetReference()
        msg.is_q = False
        msg.is_rot = True
        msg.is_pos = True
        
        omega = 2 * np.pi * frequency
        t = self.get_clock().now().nanoseconds / 1e9 - self.init_t
        
        if self.initial_state is None:
            return None
        else:            
            pos_ref = np.array(self.initial_state.ee_pos, dtype=np.float64) + np.array([0.0, radius * (np.cos(omega * t) - 1), radius * np.sin(omega * t)], dtype=np.float64)
            rot_ref = self.initial_state.ee_rot
            msg.pos_ref = pos_ref.tolist()
            msg.rot_ref = rot_ref.tolist()

            
        return msg

    

In [3]:
rclpy.init(args=None)
controller_node = ControllerInterfaceNode()
mpc_node = MPCControllerInterfaceNode()

In [ ]:
## Set Controller 
controller_node.update_gain(True)
controller_name = "TASK_PD_FRIC"
# "TASK_PD" # "JOINT_PD" 
# "JOINT_NRIC" # "TASK_NRIC" 
# "GRAVITY_COMPENSATION" 
# "TASK_PD_FRIC" # "JOINT_PD_FRIC"
controller_node.change_conroller(controller_name)

In [ ]:
controller_node.update_gain(True)

In [5]:
## Joint Space Control 

## simulation 
# q_d = [0.0, -0.2618, 0.0, 1.833, 0.0, 1.571, 0.0]
# q_d =  [0.0, 0.386, 0.0, 2.29, 0.0, -1.07, 0.0]

## real robot 
q_d = [0.0, -0.2618, 0.0, 1.833, 0.0, 1.571, 0.0]

dq_d = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ddq_d = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

controller_node.set_desired_joint_value(q_d, dq_d, ddq_d, False)

In [12]:
import math
import time

# Original desired joint positions
# q_d_base =  [0.0, 0.386, 0.0, 2.29, 0.0, -1.07, 0.0]
q_d_base =  [0.0, -0.2618, 0.0, 1.833, 0.0, 1.571, 0.0]

# Sine wave parameters
amplitude = 0.1  # Amplitude of the sine wave (radians)
frequency = 0.2  # Frequency of the sine wave (Hz)
phase = 0.0  # Phase shift of the sine wave
duration = 10  # Duration of the simulation in seconds
dt = 0.001  # Time step for the loop

# Total number of iterations
iterations = int(duration / dt)

for i in range(iterations):
    # Calculate the current time
    t = i * dt
    
    # Calculate the sine wave value
    sine_wave = amplitude * math.sin(2 * math.pi * frequency * t + phase)
    
    # Update the desired joint positions with the sine wave offset
    q_d = [q_base - sine_wave for q_base in q_d_base]
    
    # Set dq_d and ddq_d to zero
    dq_d = [0.0] * 7
    ddq_d = [0.0] * 7
    
    # Send the updated desired joint values to the controller
    controller_node.set_desired_joint_value(q_d, dq_d, ddq_d, True)
    
    # Wait for the next time step
    time.sleep(dt)

In [6]:
## Task Space Control 

controller_node.update_gain(True)
controller_name = "TASK_PD_FRIC"
 # "TASK_PD" # "JOINT_PD" 
# "JOINT_NRIC" # "TASK_NRIC" 
# "GRAVITY_COMPENSATION" 
# "TASK_PD_FRIC" # "JOINT_PD_FRIC"
controller_node.change_conroller(controller_name)

## simulation 
pos_d = [0.466941, -0.024872,  0.387849] # initial pose
rot_d = [0.0, 0.0, -1.0,
         0.0, 1.0, 0.0,
         1.0, 0.0, 0.0]
        # 1st  2nd  3rd
quat_d = [] # [w, x, y, z]

## real robot
# pos_d = [0.202915, -0.0170032,  0.51977] # new initial pose
# rot_d = [0.0, -1.0, 0.0,
#          -1.0, 0.0, 0.0,
#          0.0, 0.0, -1.0]
# quat_d = []

controller_node.set_desired_task_value(pos_d, rot_d, quat_d, True)

In [11]:
## Set circular trajectory
dt = 0.001
# frequency = 4.3
frequency = 2.68
angular_velocity = 2 * np.pi / frequency
radius = 0.10
angle = 0.0
t = 0.0

# simulation 
initial_pos = [0.466941, -0.024872,  0.387849 - radius]
rot_d = [0.0, 0.0, -1.0,
         0.0, 1.0, 0.0,
         1.0, 0.0, 0.0]

## real robot
# initial_pos = [0.2039, -0.02458,  0.52 - radius] # PD-TYPE
# initial_pos = [0.203515, -0.02463, 0.5198 - radius] # L1-TYPE
# initial_pos = [0.203175, -0.0247781, 0.519453 - radius] # NON
# rot_d = [0.0, -1.0, 0.0,
#          -1.0, 0.0, 0.0,
#          0.0, 0.0, -1.0]
quat_d = []

while t <= frequency:
    x = initial_pos[0] 
    y = initial_pos[1] + radius * np.sin(angle)
    z = initial_pos[2] + radius * np.cos(angle)
    pose_d = [x, y, z]
    quat_d = []

    controller_node.set_desired_task_value(pose_d, rot_d, quat_d, True)

    angle += angular_velocity * dt
    if angle > 2 * np.pi:
        angle -= 2 * np.pi

    time.sleep(dt)
    t += dt

In [ ]:
## Set Lissajous trajectory
dt = 0.001
frequency = 20.0
angular_velocity = 2 * np.pi / frequency
radius = 0.1
angle = 0.0
t = 0.0

## simulation 
# initial_pos = [0.466941, -0.024872,  0.387849 - radius]
# rot_d = [0.0, 0.0, -1.0,
#          0.0, 1.0, 0.0,
#          1.0, 0.0, 0.0]

## real robot
initial_pos = [0.205, 0.0,  0.535 - radius] # new initial pose
rot_d = [-1.0, 0.0, 0.0,
         0.0, 1.0, 0.0,
         0.0, 0.0, -1.0]
quat_d = []

while True:
    x = initial_pos[0]
    y = initial_pos[1] + radius * np.sin(2 * angle)
    z = initial_pos[2] + radius * np.cos(angle)

    pose_d = [x, y, z]
    quat_d = []

    controller_node.set_desired_task_value(pose_d, rot_d, quat_d, True)

    angle += angular_velocity * dt
    if angle > 2 * np.pi:
        angle -= 2 * np.pi

    t += dt
    if t > frequency:
        break
    time.sleep(dt)